In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Hakbang 1: Tukuyin ang mga Modelong Pydantic para sa Nakabalangkas na Output

Ang mga modelong ito ang nagtatakda ng **schema** na ibabalik ng mga ahente. Ang paggamit ng `response_format` kasama ang Pydantic ay nagbibigay ng:
- ✅ Ligtas na pagkuha ng data batay sa uri
- ✅ Awtomatikong beripikasyon
- ✅ Walang error sa pag-parse mula sa mga sagot na malayang teksto
- ✅ Madaling kondisyonal na pag-ruta batay sa mga field


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Hakbang 2: Gumawa ng Hotel Booking Tool

Ang tool na ito ang tatawagin ng **availability_agent** para suriin kung may mga available na kwarto. Ginagamit natin ang `@ai_function` decorator upang:
- I-convert ang isang Python function sa isang tool na maaaring tawagin ng AI
- Awtomatikong bumuo ng JSON schema para sa LLM
- Pangasiwaan ang pag-validate ng mga parameter
- Paganahin ang awtomatikong pagtawag ng mga ahente

Para sa demo na ito:
- **Stockholm, Seattle, Tokyo, London, Amsterdam** → May mga kwarto ✅
- **Lahat ng iba pang lungsod** → Walang kwarto ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Hakbang 3: Tukuyin ang mga Function ng Kondisyon para sa Routing

Ang mga function na ito ay sinusuri ang sagot ng ahente at tinutukoy kung aling landas ang tatahakin sa workflow.

**Pangunahing Pattern:**
1. Suriin kung ang mensahe ay `AgentExecutorResponse`
2. I-parse ang structured output (Pydantic model)
3. Magbalik ng `True` o `False` upang kontrolin ang routing

Susuriin ng workflow ang mga kondisyong ito sa **mga edge** upang magpasya kung aling executor ang susunod na tatawagin.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## Hakbang 4: Gumawa ng Custom Display Executor

Ang mga executor ay mga bahagi ng workflow na gumaganap ng mga pagbabago o side effects. Ginagamit natin ang `@executor` decorator upang lumikha ng isang custom na executor na nagpapakita ng huling resulta.

**Pangunahing Konsepto:**
- `@executor(id="...")` - Nirehistro ang isang function bilang workflow executor
- `WorkflowContext[Never, str]` - Mga type hint para sa input/output
- `ctx.yield_output(...)` - Nagbibigay ng huling resulta ng workflow


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## Hakbang 5: I-load ang mga Environment Variable

I-configure ang LLM client. Ang halimbawang ito ay gumagana sa:
- **GitHub Models** (Libreng tier gamit ang GitHub token)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## Hakbang 6: Gumawa ng AI Agents na may Nakabalangkas na Output

Gumagawa tayo ng **tatlong espesyal na ahente**, bawat isa ay nakabalot sa isang `AgentExecutor`:

1. **availability_agent** - Sinusuri ang availability ng hotel gamit ang tool
2. **alternative_agent** - Nagmumungkahi ng alternatibong mga lungsod (kapag walang kwarto)
3. **booking_agent** - Hinihikayat ang pag-book (kapag may mga kwarto)

**Pangunahing Katangian:**
- `tools=[hotel_booking]` - Nagbibigay ng tool sa ahente
- `response_format=PydanticModel` - Pinipilit ang nakabalangkas na JSON na output
- `AgentExecutor(..., id="...")` - Binabalot ang ahente para sa paggamit sa workflow


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Hakbang 7: Bumuo ng Workflow gamit ang Kondisyunal na Mga Edge

Ngayon gagamitin natin ang `WorkflowBuilder` upang buuin ang graph na may kondisyunal na pag-ruta:

**Struktura ng Workflow:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**Mga Pangunahing Metodo:**
- `.set_start_executor(...)` - Itinatakda ang panimulang punto
- `.add_edge(from, to, condition=...)` - Nagdaragdag ng kondisyunal na edge
- `.build()` - Tinatapos ang workflow


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## Hakbang 8: Patakbuhin ang Test Case 1 - Lungsod na WALANG Availability (Paris)

Subukan natin ang landas ng **walang availability** sa pamamagitan ng paghingi ng mga hotel sa Paris (na walang mga kuwarto sa ating simulation).


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## Hakbang 9: Patakbuhin ang Test Case 2 - Lungsod NA May Availability (Stockholm)

Ngayon, subukan natin ang landas ng **availability** sa pamamagitan ng paghingi ng mga hotel sa Stockholm (na may mga kwarto sa ating simulation).


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## Mga Pangunahing Puntos at Susunod na Hakbang

### ✅ Mga Natutunan Mo:

1. **WorkflowBuilder Pattern**
   - Gamitin ang `.set_start_executor()` para tukuyin ang entry point
   - Gamitin ang `.add_edge(from, to, condition=...)` para sa conditional routing
   - Tawagin ang `.build()` para tapusin ang workflow

2. **Conditional Routing**
   - Ang mga condition function ay nag-iinspeksyon sa `AgentExecutorResponse`
   - I-parse ang structured outputs para makagawa ng routing decisions
   - Mag-return ng `True` para i-activate ang edge, `False` para laktawan ito

3. **Tool Integration**
   - Gamitin ang `@ai_function` para gawing AI tools ang mga Python function
   - Awtomatikong tinatawag ng mga agents ang tools kapag kinakailangan
   - Ang mga tools ay nagre-return ng JSON na maaaring i-parse ng agents

4. **Structured Outputs**
   - Gamitin ang Pydantic models para sa type-safe na pagkuha ng data
   - Itakda ang `response_format=MyModel` kapag gumagawa ng agents
   - I-parse ang mga response gamit ang `Model.model_validate_json()`

5. **Custom Executors**
   - Gamitin ang `@executor(id="...")` para gumawa ng mga workflow component
   - Ang mga executor ay maaaring mag-transform ng data o mag-perform ng side effects
   - Gamitin ang `ctx.yield_output()` para mag-produce ng workflow results

### 🚀 Mga Aplikasyon sa Totoong Buhay:

- **Pag-book ng Biyahe**: Mag-check ng availability, magmungkahi ng alternatibo, magkumpara ng mga opsyon
- **Customer Service**: Mag-route base sa uri ng isyu, damdamin, o prayoridad
- **E-commerce**: Mag-check ng imbentaryo, magmungkahi ng alternatibo, magproseso ng mga order
- **Content Moderation**: Mag-route base sa toxicity scores, user flags
- **Approval Workflows**: Mag-route base sa halaga, user role, o antas ng panganib
- **Multi-stage Processing**: Mag-route base sa kalidad ng data, pagiging kumpleto

### 📚 Susunod na Hakbang:

- Magdagdag ng mas kumplikadong kondisyon (maramihang pamantayan)
- Magpatupad ng mga loop gamit ang workflow state management
- Magdagdag ng sub-workflows para sa reusable na mga component
- Mag-integrate sa mga totoong API (hotel booking, inventory systems)
- Magdagdag ng error handling at fallback paths
- I-visualize ang workflows gamit ang built-in na visualization tools



---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, mangyaring tandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na dulot ng paggamit ng pagsasaling ito.
